In [178]:
# import required libraries

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, make_scorer

In [7]:
# Read Data for Train

# X Train
gesture_x_train = pd.read_csv("HW3-data/uWaveGestureLibrary_X_TRAIN", header=None, sep=" ",
                             prefix="xt", skipinitialspace=True)
gesture_x_train.rename(columns={"xt0":"class"}, inplace=True)
gesture_x_train["class"] = gesture_x_train["class"].astype("int").astype("category")

# Y Train
gesture_y_train = pd.read_csv("HW3-data/uWaveGestureLibrary_Y_TRAIN", header=None, sep=" ",
                             prefix="yt", skipinitialspace=True)
gesture_y_train.rename(columns={"yt0":"class"}, inplace=True)
gesture_y_train["class"] = gesture_y_train["class"].astype("int").astype("category")
gesture_y_train.drop(["class"], axis=1, inplace=True)

# Z Train
gesture_z_train = pd.read_csv("HW3-data/uWaveGestureLibrary_Z_TRAIN", header=None, sep=" ",
                             prefix="zt", skipinitialspace=True)
gesture_z_train.rename(columns={"zt0":"class"}, inplace=True)
gesture_z_train["class"] = gesture_z_train["class"].astype("int").astype("category")
gesture_z_train.drop(["class"], axis=1, inplace=True)

In [21]:
# concatenate the variables for train
concat_gesture_train = pd.concat([gesture_x_train, gesture_y_train, gesture_z_train], join="inner", axis=1)
X_train = concat_gesture_train.drop("class", axis=1)
y_train = concat_gesture_train["class"]
X_train.head()

,xt1,xt2,xt3,xt4,xt5,xt6,xt7,xt8,xt9,xt10,...,zt306,zt307,zt308,zt309,zt310,zt311,zt312,zt313,zt314,zt315
0,-0.304243,-0.304243,-0.304243,-0.304243,-0.304243,-0.304243,-0.304243,-0.304243,-0.304243,-0.304243,...,0.523217,0.513994,0.503481,0.492967,0.474522,0.456077,0.437632,0.419187,0.400743,0.382298
1,1.627311,1.627311,1.627311,1.627311,1.627311,1.627311,1.627311,1.627311,1.627311,1.627311,...,-0.427010,-0.427010,-0.427010,-0.427172,-0.428773,-0.440720,-0.452667,-0.464613,-0.476560,-0.488507
2,0.661277,0.661277,0.661277,0.661277,0.661277,0.661277,0.661277,0.661277,0.661277,0.661277,...,-0.862717,-0.862717,-0.862717,-0.862717,-0.862717,-0.862717,-0.862717,-0.862717,-0.862717,-0.862717
3,0.005185,0.005185,0.005185,0.005185,0.005185,0.005185,0.005185,0.005185,0.005185,0.005185,...,-0.187384,-0.123549,-0.055870,0.011808,0.079487,0.157056,0.253740,0.445503,0.648538,0.851573
4,1.286198,1.286198,1.286198,1.286198,1.286198,1.286198,1.286198,1.286198,1.286198,1.286198,...,1.867473,1.834319,1.756000,1.638582,1.521164,1.453266,1.515219,1.632637,1.750054,1.867473


In [4]:
# Read Data for Test

# X Test
gesture_x_test = pd.read_csv("HW3-data/uWaveGestureLibrary_X_TEST", header=None, sep=" ",
                             prefix="xt", skipinitialspace=True)
gesture_x_test.rename(columns={"xt0":"class"}, inplace=True)
gesture_x_test["class"] = gesture_x_test["class"].astype("int").astype("category")

# Y Test
gesture_y_test = pd.read_csv("HW3-data/uWaveGestureLibrary_Y_TEST", header=None, sep=" ",
                             prefix="yt", skipinitialspace=True)
gesture_y_test.rename(columns={"yt0":"class"}, inplace=True)
gesture_y_test["class"] = gesture_y_test["class"].astype("int").astype("category")
gesture_y_test.drop(["class"], axis=1, inplace=True)

# Z Test
gesture_z_test = pd.read_csv("HW3-data/uWaveGestureLibrary_Z_TEST", header=None, sep=" ",
                             prefix="zt", skipinitialspace=True)
gesture_z_test.rename(columns={"zt0":"class"}, inplace=True)
gesture_z_test["class"] = gesture_z_test["class"].astype("int").astype("category")
gesture_z_test.drop(["class"], axis=1, inplace=True)

In [22]:
# concatenate the variables for test
concat_gesture_test = pd.concat([gesture_x_test, gesture_y_test, gesture_z_test], join="inner", axis=1)
X_test = concat_gesture_test.drop("class", axis=1)
y_test = concat_gesture_test["class"]
X_test.head()

,xt1,xt2,xt3,xt4,xt5,xt6,xt7,xt8,xt9,xt10,...,zt306,zt307,zt308,zt309,zt310,zt311,zt312,zt313,zt314,zt315
0,1.557018,1.557018,1.557018,1.557018,1.557018,1.557018,1.557018,1.557018,1.557018,1.557018,...,-0.348423,-0.348423,-0.344675,-0.340728,-0.333147,-0.325423,-0.317700,-0.309976,-0.302252,-0.294528
1,-0.075096,-0.075096,-0.075096,-0.075096,-0.075096,-0.075096,-0.075096,-0.075096,-0.075096,-0.075096,...,0.961350,0.849142,0.725546,0.589277,0.539075,0.515810,0.515810,0.515810,0.515810,0.515810
2,1.509012,1.526296,1.543580,1.560865,1.573811,1.576913,1.578592,1.578592,1.578592,1.578592,...,-0.565962,-0.475680,-0.399089,-0.344074,-0.289058,-0.234042,-0.179026,-0.124011,-0.068995,-0.013979
3,1.303974,1.303974,1.303974,1.303974,1.303974,1.303974,1.303974,1.303974,1.303974,1.303974,...,1.044950,1.044950,1.044950,1.044950,1.079339,1.115085,1.171001,1.227442,1.283883,1.340324
4,1.298089,1.298089,1.298089,1.298089,1.298089,1.298089,1.298089,1.298089,1.298089,1.298089,...,-0.693460,-0.689765,-0.686070,-0.682375,-0.678680,-0.674985,-0.671290,-0.667595,-0.663900,-0.660205


# TASK 1 - Nearest Neighbors Applications on the Gesture Data

### a) 

In [14]:
# Two distance measures will be used for the KNN:
# Euclidean Distance, because it is the most used distance measure (default measure of the algorith)
# Manhattan Distance, because it is said to perform better on a high dimensional data.

In [34]:
knn_l1 = KNeighborsClassifier(p=1) # n-jobs allows the computer to use all the processors.
knn_l2 = KNeighborsClassifier(p=2) # n-jobs allows the computer to use all the processors.

In [44]:
param_grid = {"n_neighbors": list(np.arange(1,21,1))}
l1_cv_model = GridSearchCV(knn_l1, cv=10, n_jobs=-1, verbose=2, param_grid= param_grid)
l2_cv_model = GridSearchCV(knn_l2, cv=10, n_jobs=-1, verbose=2, param_grid= param_grid)

In [47]:
l1_cv_model.fit(X_train, y_train)
l1_cv_model.best_params_

Fitting 10 folds for each of 20 candidates, totalling 200 fits


{'n_neighbors': 1}

In [48]:
l2_cv_model.fit(X_train, y_train)
l2_cv_model.best_params_

Fitting 10 folds for each of 20 candidates, totalling 200 fits


{'n_neighbors': 3}

### For Manhattan Distance, k=1 gives the least error.
### For Euclidean Distance, k=3 gives the least error.

### b)

In [73]:
y_pred_l1 = l1_cv_model.best_estimator_.predict(X_test)
y_pred_l2 = l2_cv_model.best_estimator_.predict(X_test)

In [94]:
confusion_df_l1 = pd.DataFrame({"Actual":y_test, "Predicted":y_pred_l1})
print("Confusion Matrix For KNN with l1-norm and k=1")
c_matrix_l1 = pd.crosstab(confusion_df_l1["Actual"], confusion_df_l1["Predicted"])
c_matrix_l1

Confusion Matrix For KNN with l1-norm and k=1


Predicted,1,2,3,4,5,6,7,8
Actual,,,,,,,,
1,430,0,0,2,0,5,0,0
2,1,451,0,0,0,0,0,0
3,2,0,418,0,16,14,4,0
4,3,0,0,392,40,11,0,4
5,3,0,9,5,415,1,0,0
6,3,0,5,13,16,411,0,1
7,0,0,2,0,0,0,445,0
8,0,0,0,3,1,0,0,456


In [107]:
true_pred_l1=0
for i in range(0,8):
    true_pred_l1+= c_matrix_l1.iloc[i].iloc[i]
acc_score_l2 = true_pred_l1/c_matrix_l1.sum().sum()
print("Accuracy Score for KNN with L1 Norm is:", acc_score_l2)

Accuracy Score for KNN with L1 Norm is: 0.954215522054718


In [105]:
confusion_df_l2 = pd.DataFrame({"Actual":y_test, "Predicted":y_pred_l2})
print("Confusion Matrix For KNN with l2-norm and k=3")
c_matrix_l2 = pd.crosstab(confusion_df_l2["Actual"], confusion_df_l2["Predicted"])
c_matrix_l2

Confusion Matrix For KNN with l2-norm and k=3


Predicted,1,2,3,4,5,6,7,8
Actual,,,,,,,,
1,431,0,0,2,0,4,0,0
2,1,449,0,0,0,0,2,0
3,2,0,413,0,15,20,4,0
4,7,0,0,370,60,6,0,7
5,3,0,6,1,422,1,0,0
6,6,0,7,15,28,392,1,0
7,0,0,1,0,0,0,446,0
8,0,0,0,1,1,0,0,458


In [104]:
true_pred_l2=0
for i in range(0,8):
    true_pred_l2+= c_matrix_l2.iloc[i].iloc[i]
acc_score_l2 = true_pred/c_matrix_l2.sum().sum()
print("Accuracy Score for KNN with L2 Norm is:", acc_score_l2)

Accuracy Score for KNN with L2 Norm is: 0.9438860971524288


### c)

### I think that it makes more sense to weight x and y axis more than the z-axis because gestures are defined by how they accelerate at these axes (Which can be see on the Figure 1 of the HW3 instructions). Information of the z axis is more like an extra information used due to the fact that people do not present gestures perpendicular to their devices.

# TASK 2 - Logistic Regression Applications on the Gesture Data

In [119]:
# Convert targets to indicate if the class is 3 or not.
y_train_class3 = (y_train==3)*1
y_test_class3 = (y_test==3)*1

### a)

In [135]:
# Find the ratio of Class 3 instances in the training data
ratio_class3 = y_train_class3.sum()/len(y_train)
ratio_class3

0.11830357142857142

In [126]:
lr_model = LogisticRegression(max_iter=100000)
lr_model.fit(X_train, y_train_class3)

LogisticRegression(max_iter=100000)

In [146]:
y_pred_prob = lr_model.predict_proba(X_test)
y_pred_lr = (y_pred_prob[:,1]>ratio_class3)*1

In [147]:
confusion_lr = pd.DataFrame({"Actual":y_test_class3, "Predicted":y_pred_lr})
print("Confusion Matrix For Logistic Regression on Class3")
c_matrix_lr = pd.crosstab(confusion_lr["Actual"], confusion_lr["Predicted"])
c_matrix_lr

Confusion Matrix For Logistic Regression on Class3


Predicted,0,1
Actual,,
0,2925,203
1,95,359


### Accuracy is (2925+359)/(2925+359+203+95) = 0.917

### b)

In [161]:
lr_lasso_model = LogisticRegression(max_iter=100000, penalty='l1', solver='liblinear')

In [179]:
param_grid = {'C': [0.0001, 0.001, 0.05, 0.01, 0.5, 0.1, 1, 5, 10, 100]}
LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)
lr_lasso_cv_model = GridSearchCV(lr_lasso_model, cv=10, n_jobs=-1, verbose=2, param_grid=param_grid, scoring=LogLoss,
                                error_score='raise')
lr_lasso_cv_model.fit(X_train, y_train_class3)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


GridSearchCV(cv=10, error_score='raise',
             estimator=LogisticRegression(max_iter=100000, penalty='l1',
                                          solver='liblinear'),
             n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.05, 0.01, 0.5, 0.1, 1, 5, 10,
                               100]},
             scoring=make_scorer(log_loss, greater_is_better=False, needs_proba=True),
             verbose=2)

In [180]:
lr_lasso_cv_model.best_params_

{'C': 0.5}

### We find that best C value is 0.5 . It is defined as 'Inverse of Regularization Strength', therefore lambda= 2 is our parameter.

In [183]:
lr_lasso_tuned = lr_lasso_cv_model.best_estimator_
y_pred_prob = lr_lasso_tuned.predict_proba(X_test)
y_pred_lr_lasso = (y_pred_prob[:,1]>ratio_class3)*1

In [189]:
confusion_lr_lasso = pd.DataFrame({"Actual":y_test_class3, "Predicted":y_pred_lr_lasso})
print("Confusion Matrix For Logistic Regression w/Lasso on Class3")
c_matrix_lr_lasso = pd.crosstab(confusion_lr_lasso["Actual"], confusion_lr_lasso["Predicted"])
c_matrix_lr_lasso

Confusion Matrix For Logistic Regression w/Lasso on Class3


Predicted,0,1
Actual,,
0,2873,255
1,32,422


### Accuracy is (2873+422)/(2873+422+255+32) = 0.92

### Logistic Regression model with Lasso regularization has a better accuracy score and it is better at predicting the Class3 gestures. However, Logistic Regression in part a) is better at detecting the observations that are not Class 3.

In [197]:
# Print the coefficients of the LR with Lasso penalty, ones that have positive values.
lr_lasso_coefs = pd.DataFrame({"Coef":lr_lasso_tuned.coef_[0]}, index=X_train.columns)
lr_lasso_coefs[lr_lasso_coefs["Coef"]>0]

,Coef
xt127,0.043646
xt128,0.020288
xt208,0.087571
xt235,0.118792
xt236,0.279351
xt237,0.373758
xt238,0.160198
xt248,0.076753
xt249,0.151164
xt258,0.399740


### We see that nearly half of the significant features are from the x-axis. It makes sense because if we look at the shape of the class 3 gesture, it is a straight line from left to right. Therefore, we mostly need x-axis coordinate to define it. Also, another interesting thing is that initial timepoints are not significant (s.t. xt127, yt160 and zt87 are the earliest timepoints for the respective axes) at determining if a gesture belongs to class 3. We need y values from t=160 to t=231. In my opinion, it is because if there is no vertical movement in the middle of the gesture, it is more likely that the gesture is performed on the horizontal line (Class 3 or Class 4) and thus narrows the choice of the gestures.

### c)

In [ ]:
# This line is used to create html version of the notebook.

In [200]:
import os

os.system('jupyter nbconvert --to html HW3.ipynb')

0